In [15]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
from qiskit import *
from copy import deepcopy
from qiskit import *
from qiskit.quantum_info import state_fidelity, process_fidelity, Chi, Choi, PTM, SuperOp
from qiskit.tools.qi.qi import outer
import qiskit.ignis.mitigation.measurement as mc
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info.operators import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator

# Tomography functions
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

# Qiskit Aer noise module imports
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error, amplitude_damping_error
from qiskit.providers.aer.noise.errors import depolarizing_error, phase_damping_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

In [33]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])
#circ.measure(q[0],c[0])

noise_model = NoiseModel()

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = Choi(outer(ideal_unitary.ravel(order='F')))
chi_ideal = Chi(choi_ideal)
ptm_ideal = PTM(choi_ideal)
superop_ideal = SuperOp(choi_ideal)
# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=100)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)

# Add depolarizing error to all single qubit u1, u2, u3 gates
error = amplitude_damping_error(0.1, 0, True)
error2 = error.tensor(error)
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error2, ["cx"])

simulator = QasmSimulator()
# Run the noisy simulation
job = execute(circ, simulator,
              basis_gates=noise_model.basis_gates,
              noise_model=noise_model)
result = job.result()

meas_calibs, process_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qpt_circs, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),process_labels)

tomo_bell = ProcessTomographyFitter(job_tomo.result(), qpt_circs)
PTM_bell= PTM(tomo_bell.fit()).data

PTM_ideal = qpt_tomo.fit().data
F_bell = state_fidelity(PTM_ideal/2, PTM_bell/2)
print('Fit Fidelity (no correction) =', F_bell)
print(PTM_bell)

No classical registers in circuit "circuit1771": result data will not contain counts.
No measurements in circuit "circuit1771": count data will return all zeros.


Fit Fidelity (no correction) = 0.7264379811496932
[[ 1.00000000e+00+0.j -7.88625243e-04+0.j -4.12102054e-03+0.j
  -1.49646464e-05+0.j]
 [ 1.42659656e-05+0.j  1.13890674e-03+0.j  1.62996754e-04+0.j
   9.98682690e-01+0.j]
 [ 4.50998367e-03+0.j -1.35610547e-02+0.j -9.99300193e-01+0.j
  -3.38581603e-04+0.j]
 [-1.23604477e-03+0.j  9.98874042e-01+0.j -1.47946460e-02+0.j
  -1.50511578e-03+0.j]]


Gate fidelity** 